# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag", "earliest_cr_line", "next_pymnt_d", "last_credit_pull_d",
    "sec_app_inq_last_6mths"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,earliest_cr_line,next_pymnt_d,last_credit_pull_d,sec_app_inq_last_6mths
0,18000.0,0.1557,433.63,MORTGAGE,60000.0,Verified,Mar-2019,low_risk,n,62.80,...,196496.0,170509.0,10200.0,73796.0,N,N,Nov-2014,May-2019,Apr-2019,0.0
1,40000.0,0.2250,1537.99,MORTGAGE,54000.0,Verified,Mar-2019,low_risk,n,59.09,...,123858.0,107194.0,19100.0,64858.0,N,N,Jan-2003,May-2019,Apr-2019,0.0
2,13975.0,0.1640,342.83,RENT,67000.0,Verified,Mar-2019,low_risk,n,13.04,...,545200.0,12322.0,9600.0,9200.0,N,N,Jul-2004,May-2019,Apr-2019,0.0
3,40000.0,0.1171,883.93,MORTGAGE,79000.0,Verified,Mar-2019,low_risk,n,21.89,...,520784.0,111899.0,76700.0,71224.0,N,N,Aug-2005,May-2019,Apr-2019,0.0
4,35000.0,0.0702,1081.02,OWN,50000.0,Not Verified,Mar-2019,low_risk,n,38.93,...,135831.0,68399.0,32100.0,85531.0,N,N,Nov-2003,May-2019,Apr-2019,4.0


# Split the Data into Training and Testing

In [7]:
#Create Datafram

df_training_variables = pd.get_dummies(df, columns=[
    'initial_list_status',
    'home_ownership',
    'verification_status',
    'issue_d',
    'pymnt_plan',
    'application_type',
    'hardship_flag',
    'debt_settlement_flag'])
df_training_variables

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,18000.0,0.1557,433.63,60000.0,low_risk,62.80,0.0,0.0,9.0,0.0,...,0,0,1,0,0,1,1,1,1,1
1,40000.0,0.2250,1537.99,54000.0,low_risk,59.09,0.0,0.0,19.0,0.0,...,0,0,1,0,0,1,1,1,1,1
2,13975.0,0.1640,342.83,67000.0,low_risk,13.04,0.0,0.0,6.0,0.0,...,0,0,1,0,0,1,1,1,1,1
3,40000.0,0.1171,883.93,79000.0,low_risk,21.89,1.0,0.0,29.0,0.0,...,0,0,1,0,0,1,1,1,1,1
4,35000.0,0.0702,1081.02,50000.0,low_risk,38.93,0.0,0.0,14.0,0.0,...,1,0,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9606,40000.0,0.1894,1465.03,55000.0,low_risk,18.08,0.0,1.0,11.0,0.0,...,0,0,1,0,1,0,1,1,1,1
9607,40000.0,0.2235,1112.74,100000.0,low_risk,46.31,4.0,0.0,13.0,0.0,...,0,1,0,0,1,0,1,1,1,1
9608,40000.0,0.1033,1296.90,100000.0,low_risk,28.67,0.0,0.0,15.0,1.0,...,0,0,1,0,1,0,1,1,1,1
9609,20000.0,0.1502,476.01,54000.0,low_risk,48.16,0.0,1.0,13.0,0.0,...,0,0,1,0,1,0,1,1,1,1


In [8]:
# Create our features
X = df_training_variables.drop(columns="loan_status", axis=1)

# Create our target
y = df["loan_status"]
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,earliest_cr_line,next_pymnt_d,last_credit_pull_d,sec_app_inq_last_6mths
0,18000.0,0.1557,433.63,MORTGAGE,60000.0,Verified,Mar-2019,low_risk,n,62.80,...,196496.0,170509.0,10200.0,73796.0,N,N,Nov-2014,May-2019,Apr-2019,0.0
1,40000.0,0.2250,1537.99,MORTGAGE,54000.0,Verified,Mar-2019,low_risk,n,59.09,...,123858.0,107194.0,19100.0,64858.0,N,N,Jan-2003,May-2019,Apr-2019,0.0
2,13975.0,0.1640,342.83,RENT,67000.0,Verified,Mar-2019,low_risk,n,13.04,...,545200.0,12322.0,9600.0,9200.0,N,N,Jul-2004,May-2019,Apr-2019,0.0
3,40000.0,0.1171,883.93,MORTGAGE,79000.0,Verified,Mar-2019,low_risk,n,21.89,...,520784.0,111899.0,76700.0,71224.0,N,N,Aug-2005,May-2019,Apr-2019,0.0
4,35000.0,0.0702,1081.02,OWN,50000.0,Not Verified,Mar-2019,low_risk,n,38.93,...,135831.0,68399.0,32100.0,85531.0,N,N,Nov-2003,May-2019,Apr-2019,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9606,40000.0,0.1894,1465.03,MORTGAGE,55000.0,Verified,Jan-2019,low_risk,n,18.08,...,211528.0,36258.0,14200.0,25428.0,N,N,Sep-1993,May-2019,Apr-2019,1.0
9607,40000.0,0.2235,1112.74,MORTGAGE,100000.0,Source Verified,Jan-2019,low_risk,n,46.31,...,570818.0,203584.0,50500.0,237068.0,N,N,Dec-2004,May-2019,Apr-2019,0.0
9608,40000.0,0.1033,1296.90,MORTGAGE,100000.0,Verified,Jan-2019,low_risk,n,28.67,...,480699.0,244719.0,31900.0,222999.0,N,N,Sep-2006,May-2019,Apr-2019,0.0
9609,20000.0,0.1502,476.01,RENT,54000.0,Verified,Jan-2019,low_risk,n,48.16,...,162382.0,122416.0,18000.0,138882.0,N,N,Nov-1996,May-2019,Apr-2019,1.0


In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,...,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.000000,9611.0,9611.0,9611.0,9611.0
mean,21224.253460,0.134981,598.603060,71127.468760,35.034184,0.207054,0.431069,12.304547,0.121736,18820.379669,...,0.370825,0.265113,0.364062,0.392155,0.415670,0.192176,1.0,1.0,1.0,1.0
std,11121.417441,0.049900,321.040161,50783.809018,48.054925,0.676033,0.710489,5.796924,0.329533,22504.004634,...,0.483051,0.441416,0.481191,0.488256,0.492863,0.394031,0.0,0.0,0.0,0.0
min,1000.000000,0.064600,31.430000,40.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0
25%,12000.000000,0.103300,342.485000,40500.000000,18.310000,0.000000,0.000000,8.000000,0.000000,6260.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0
50%,20000.000000,0.129800,545.830000,60000.000000,27.460000,0.000000,0.000000,11.000000,0.000000,12950.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0
75%,30000.000000,0.164000,812.070000,87000.000000,40.200000,0.000000,1.000000,15.000000,0.000000,23842.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0,1.0
max,40000.000000,0.301700,1676.230000,950000.000000,999.000000,14.000000,5.000000,69.000000,2.000000,321859.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     9558
high_risk      53
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(7208, 97)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [14]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

RandOverSamp = RandomOverSampler(random_state=1)
X_resampled, y_resampled = RandOverSamp.fit_resample(X_train, y_train)

Counter(y_resampled)

KeyError: 'Only the Series name can be used for the key in Series dtype mappings.'

In [ ]:
#FloatX = float(X_resampled)
#Floaty = float(y_resampled)

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled) 

y_pred = model.predict(X_test)

results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

NameError: name 'X_resampled' is not defined

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

### SMOTE Oversampling

In [ ]:
# Resample the training data with SMOTE
# YOUR CODE HERE

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE